In [80]:
pip install -r requirements.txt

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [81]:
from llama_index.readers.obsidian import ObsidianReader
from llama_index.core.memory.chat_memory_buffer import MessageRole
from llama_index.core import SimpleDirectoryReader, KnowledgeGraphIndex, VectorStoreIndex
from llama_index.core.graph_stores import SimpleGraphStore
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core import Document, PropertyGraphIndex
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore
from llama_index.core import Settings
from IPython.display import Markdown, display
from llama_index.llms.ollama import Ollama
from tqdm.notebook import tqdm
import time
import os
from llama_index.core.llms import ChatMessage
from llama_index.core import StorageContext
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.memory import ChatMemoryBuffer
import logging
import sys
import ipywidgets as widgets
import json
from llama_index.core.callbacks import CallbackManager
from llama_index.core.callbacks import LlamaDebugHandler
from llama_index.core import ServiceContext
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.retrievers import KnowledgeGraphRAGRetriever
from llama_index.core.indices.property_graph import (
    SimpleLLMPathExtractor,
    SchemaLLMPathExtractor,
    DynamicLLMPathExtractor,
)
import yaml
import networkx as nx
from pyvis.network import Network
from dotenv import load_dotenv


In [82]:
from llama_index.core import (
    load_index_from_storage,
    load_indices_from_storage,
    load_graph_from_storage,
)

In [83]:
import nest_asyncio

nest_asyncio.apply()

In [84]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

# Set LLM (OpenAI)

In [85]:
load_dotenv()  # Charge les variables depuis le fichier .env
api_key = os.getenv("OPENAI_API_KEY_UP")
# Modifier ou ajouter une variable d'environnement
os.environ["OPENAI_API_KEY"] = api_key

In [86]:
llm = OpenAI(temperature=0, model="gpt-4o", max_tokens=3000)
Settings.llm = llm
Settings.chunk_size = 512

# Set local LLM for embeddings

In [87]:
# bge-base embedding model
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5")
#Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-small")

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
Load pretrained SentenceTransformer: BAAI/bge-base-en-v1.5
INFO:sentence_transformers.SentenceTransformer:2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']
2 prompts are loaded, with the keys: ['query', 'text']


# Set LLM for chat  (Local)

In [88]:
#llm = Ollama(model="tinyllama", request_timeout=120.0)
#Settings.llm = llm
#Settings.chunk_size = 512

# Test LLM

In [89]:
messages = [
    ChatMessage(
        role="system", content="You are a data governance consultant"
    ),
    ChatMessage(role="user", content="What's your favorite data tool ?"),
]
resp = llm.chat(messages)
print(resp)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
assistant: As a data governance consultant, I don't have personal preferences or feelings, but I can certainly highlight some popular data tools that are widely appreciated in the industry for their capabilities. Tools like Collibra, Informatica, and Alation are often favored for their robust data governance features, including data cataloging, data lineage, and compliance management. Each tool has its strengths, and the best choice often depends on the specific needs and context of an organization.


# Load storage contexts

## Load vector storage context

In [90]:
vector_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="vector"),
    vector_store=SimpleVectorStore.from_persist_dir(
        persist_dir="vector"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="vector"),
)

## Load knowledge graph storage context

In [91]:
graph_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="knowledge_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="knowledge_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="knowledge_graph"),
)

## Load onto graph storage context


In [92]:
onto_storage_context = StorageContext.from_defaults(
    docstore=SimpleDocumentStore.from_persist_dir(persist_dir="onto_graph"),
    graph_store=SimpleGraphStore.from_persist_dir(
        persist_dir="onto_graph"
    ),
    index_store=SimpleIndexStore.from_persist_dir(persist_dir="onto_graph"),
)

In [93]:
print(onto_storage_context)

StorageContext(docstore=<llama_index.core.storage.docstore.simple_docstore.SimpleDocumentStore object at 0x163503c40>, index_store=<llama_index.core.storage.index_store.simple_index_store.SimpleIndexStore object at 0x1635019c0>, vector_stores={'default': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={})), 'image': SimpleVectorStore(stores_text=False, is_embedding_query=True, data=SimpleVectorStoreData(embedding_dict={}, text_id_to_ref_doc_id={}, metadata_dict={}))}, graph_store=<llama_index.core.graph_stores.simple.SimpleGraphStore object at 0x163502a70>, property_graph_store=None)


# Load index

## Load vector index

In [94]:
simple_index = load_index_from_storage(vector_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


### Set retriever

In [95]:
simple_query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

### Test retriever

In [96]:
simple_rag_retriever = simple_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = simple_query_engine.query(
    "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [97]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pouvez impliquer plusieurs personnes et équipes clés :

1. **Mme Laurent** : Elle a exprimé le besoin d'accéder à des données plus complètes et en temps réel. Vous pouvez l'impliquer pour définir les besoins en données et les indicateurs clés de performance (KPIs) à suivre.

2. **M. Dupont** : En tant que Directeur de la production, il peut être crucial pour assurer la complétude des champs dans le Datawarehouse. Il peut également aider à identifier les champs critiques et prioritaires, et à mettre en place des contrôles automatiques pour garantir la qualité des données.

3. **Équipes IT** : Elles seront essentielles pour la mise en place des processus ETL, la validation des données, et la gestion de la sécurité et des sauvegardes de la base de données.

4. **Équipes Opérationnelles** : Elles devraient être formées pour comprendre l'importance de la saisie complète des données et pour collaborer avec les équipes IT afin de définir les priorités des données critiques.

5. **Responsable de la Base Opération** : Cette personne peut aider à fournir une documentation claire et détaillée de la base de données "Opérations" pour garantir une compréhension commune et faciliter son utilisation.

Ensemble, ces acteurs peuvent contribuer à la réussite de votre projet en assurant la qualité, la complétude, et l'accessibilité des données opérationnelles.</b>

## Load graph index

In [98]:
graph_index = load_index_from_storage(graph_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [99]:
nx_graph = graph_index.get_networkx_graph()

In [100]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 87


In [101]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 116 nodes and 87 edges
Nodes: ['Document', 'Aspects techniques du datawarehouse', 'Spécifications des flux de données', 'Caractéristiques techniques', 'Description', 'Datawarehouse', 'Réservoir central', 'Données consolidées', 'Élément crucial', 'Fiabilité', 'Postgresql 14', 'Base de données', 'Modèle en étoile', 'Structure', 'Directeur de la production', 'Accès', 'Politiques strictes', 'Administrateurs système', 'Indirect', 'Contrats', 'Erp', 'Système central de gestion', 'Données relatives à', 'Visibilité en temps réel', 'Interfaces', 'Transfert des données', 'Informations précises et à jour', 'Interface', 'Etl', 'Outil', 'Talend data integration', 'Source', 'Erp (sap', 'Destination', 'Tables de faits et dimensions du datawarehouse', 'Planification', 'Ordonnanceur cron', 'Alertes', "Emails et logs en cas d'échec", 'Gestion', 'Données', 'Fragmentées', 'Validation', 'Contrôles sur les types et les règles métier', 'Reporting', 'Tableau de bord sur les échecs de flux', 'Rétent

### Set retriever

In [118]:
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = graph_query_engine.query(
    "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
> Querying with idx: 08b06ef3-6916-491a-a44d-3

In [119]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pouvez impliquer plusieurs parties prenantes :

1. **Équipes opérationnelles** : Elles peuvent être formées pour comprendre les nouvelles règles et processus liés à l'utilisation des données dans le projet. Une formation est prévue pour ces équipes, ce qui peut être une bonne occasion de les impliquer.

2. **Responsables des opérations** : Ils peuvent être impliqués pour superviser l'intégration des données opérationnelles dans le projet. Par exemple, un responsable comme Jean Dupont pourrait être désigné pour suivre le projet.

3. **Équipe technique de l'ERP et du Datawarehouse** : Ces équipes peuvent être cruciales pour gérer les flux de données entre l'ERP et le Datawarehouse, en s'assurant que les données sont correctement synchronisées et intégrées.

4. **Spécialistes en ETL** : Ils peuvent être impliqués pour mettre en place des règles de validation automatique et gérer les transformations complexes des données.

5. **Gestionnaires de projets** : Ils peuvent coordonner les différentes parties prenantes et s'assurer que le projet respecte les délais et les objectifs fixés.

En impliquant ces parties, vous pouvez vous assurer que le projet est bien structuré et que les données opérationnelles sont utilisées efficacement.</b>

### Test retriever

In [103]:
graph_rag_retriever = graph_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = graph_query_engine.query(
    "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?",
)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
> Querying with idx: 08b06ef3-6916-491a-a44d-3

In [104]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pouvez impliquer plusieurs parties prenantes :

1. **Équipes opérationnelles** : Elles peuvent être formées pour comprendre les nouvelles règles et processus liés au projet. Une formation est prévue pour ces équipes, ce qui les rend essentielles pour le succès du projet.

2. **Responsables des opérations** : Ils peuvent être impliqués pour superviser et gérer les aspects opérationnels du projet. Par exemple, un responsable comme Jean Dupont pourrait être désigné pour suivre le projet.

3. **Équipe technique de l'ERP et du Datawarehouse** : Ces équipes sont cruciales pour assurer la synchronisation et l'intégration des données entre l'ERP et le Datawarehouse. Elles peuvent travailler sur les flux de données, les transformations nécessaires, et la mise en place de règles de validation automatique dans les processus ETL.

4. **Gestionnaires de projets** : Ils peuvent coordonner les différentes parties prenantes, gérer les délais, et s'assurer que les objectifs du projet sont atteints.

Ensemble, ces parties prenantes peuvent collaborer pour garantir que le projet est bien structuré, que les données sont correctement intégrées et que les objectifs stratégiques de l'entreprise sont atteints.</b>

## Load onto index

In [105]:
onto_storage_context = StorageContext.from_defaults(persist_dir="onto_graph")
# Load the PropertyGraphIndex from the storage context

onto_index = load_index_from_storage(onto_storage_context)

INFO:llama_index.core.indices.loading:Loading all indices.
Loading all indices.
Loading all indices.
Loading all indices.


In [106]:
onto_index.property_graph_store.save_networkx_graph(
    name="OntoGraph.html"
)

### Set retriever

In [107]:
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=8,
)

### Test retriever

In [108]:
onto_rag_retriever = onto_index.as_retriever(
    retriever_mode="hybrid",  # or "embedding" or "hybrid"
    verbose=True
)

response = onto_query_engine.query(
    "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?",
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [109]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pouvez impliquer plusieurs parties prenantes et définir des rôles spécifiques :

1. **Équipes Opérationnelles et IT** : Collaborez étroitement avec ces équipes pour définir les priorités des données critiques et assurer une intégration fluide des données dans le Datawarehouse.

2. **Responsable de Projet** : Une personne comme Jean Dupont, qui a déjà une expérience dans la gestion des opérations, pourrait être responsable de superviser le projet et de coordonner les différentes équipes.

3. **Équipe de Formation** : Organisez une session de formation pour les équipes terrain afin de les sensibiliser à l'importance de la saisie complète des données dans l'ERP, ce qui est crucial pour la qualité des données opérationnelles.

4. **Spécialistes en ETL** : Mettez en place des contrôles automatiques pour détecter et signaler les champs manquants lors des processus ETL, garantissant ainsi la qualité et la complétude des données transférées.

5. **Analystes de Données** : Ils peuvent travailler sur l'audit des données disponibles et identifier les champs pertinents pour le projet, en s'assurant que les données critiques sont bien intégrées et exploitables.

En impliquant ces parties prenantes, vous pouvez assurer une gestion efficace et une utilisation optimale de vos données opérationnelles pour le projet.</b>

# Visualize knowledge graph

In [110]:
nx_graph = graph_index.get_networkx_graph()

In [111]:
# Count the number of nodes
num_nodes = len(nx_graph.edges())

print(f"Number of nodes in the knowledge graph: {num_nodes}")

Number of nodes in the knowledge graph: 87


In [112]:
g = graph_index.get_networkx_graph()
print(g)
print("Nodes:", g.nodes())
print("Edges:", g.edges())
net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(g)

with open("knowledge_graph.html", "w", encoding="utf-8") as f:
    f.write(net.generate_html())

Graph with 116 nodes and 87 edges
Nodes: ['Document', 'Aspects techniques du datawarehouse', 'Spécifications des flux de données', 'Caractéristiques techniques', 'Description', 'Datawarehouse', 'Réservoir central', 'Données consolidées', 'Élément crucial', 'Fiabilité', 'Postgresql 14', 'Base de données', 'Modèle en étoile', 'Structure', 'Directeur de la production', 'Accès', 'Politiques strictes', 'Administrateurs système', 'Indirect', 'Contrats', 'Erp', 'Système central de gestion', 'Données relatives à', 'Visibilité en temps réel', 'Interfaces', 'Transfert des données', 'Informations précises et à jour', 'Interface', 'Etl', 'Outil', 'Talend data integration', 'Source', 'Erp (sap', 'Destination', 'Tables de faits et dimensions du datawarehouse', 'Planification', 'Ordonnanceur cron', 'Alertes', "Emails et logs en cas d'échec", 'Gestion', 'Données', 'Fragmentées', 'Validation', 'Contrôles sur les types et les règles métier', 'Reporting', 'Tableau de bord sur les échecs de flux', 'Rétent

# (Simple) Query the vectors

In [113]:
query = "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?"
query_engine = simple_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [114]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pouvez impliquer plusieurs personnes et équipes clés :

1. **Mme Laurent** : Elle a exprimé le besoin d'accéder à des données plus complètes et en temps réel. Vous pouvez l'impliquer pour définir les besoins en données et les indicateurs clés de performance (KPIs) à suivre.

2. **M. Dupont** : En tant que directeur de production, il a souligné l'importance de la complétude des champs dans le Datawarehouse. Il peut être impliqué pour s'assurer que les données sont complètes et fiables, et pour superviser la mise en place de contrôles automatiques pour détecter les champs manquants.

3. **Équipes opérationnelles** : Elles devraient être formées pour comprendre l'importance de la saisie complète des données et pour collaborer avec l'équipe IT afin de définir les priorités des données critiques.

4. **Équipe IT** : Pour mettre en place les règles de validation automatique dans les processus ETL et pour assurer la sécurité et la sauvegarde des données.

5. **Responsable de la base de données "Opérations"** : Pour garantir une documentation claire et détaillée, facilitant l'utilisation et la maintenance de la base de données.

Ces collaborations permettront de structurer efficacement votre projet data et d'optimiser l'utilisation des données opérationnelles.</b>

# (Simple) Query the knowledge graph 

In [115]:
query = "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?"
graph_query_engine = graph_index.as_query_engine(
 include_text=True,
 response_mode="tree_summarize",
 embedding_mode="hybrid",
 similarity_top_k=8,
)

response = graph_query_engine.query(query)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
> Querying with idx: 4a9725f2-a5cd-47ab-a5cf-c6ff2c957b8e: 5.2 Risques

- Panne des interfaces ETL pouvant interrompre les flux de donné...
INFO:llama_index.core.indices.knowledge_graph.retrievers:> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
> Querying with idx: 08b06ef3-6916-491a-a44d-31cf8e4fa62c: ---
entite: applicatif
feeds: "[[Datawarehouse]]"
---
Ce document détaille le...
> Querying with idx: 08b06ef3-6916-491a-a44d-3

In [116]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pouvez impliquer plusieurs parties prenantes :

1. **Équipes opérationnelles** : Elles peuvent être formées pour comprendre les nouvelles règles et processus liés au projet. Une formation est prévue pour ces équipes avant fin février 2025.

2. **Responsables des opérations** : Ils peuvent être impliqués pour superviser l'intégration des données opérationnelles dans le Datawarehouse. Par exemple, un responsable comme Jean Dupont pourrait être désigné pour gérer des opérations spécifiques.

3. **Équipe technique de l'ERP et du Datawarehouse** : Ces équipes peuvent travailler sur l'identification des champs critiques et prioritaires dans le Datawarehouse, ainsi que sur la mise en place de règles de validation automatique dans les processus ETL.

4. **Gestionnaires de projets** : Ils peuvent être impliqués pour assurer que chaque opération est correctement liée à un projet spécifique, en utilisant des champs comme `id_projet` pour maintenir la traçabilité et la gestion des données.

Ces parties prenantes joueront un rôle crucial dans la gestion, l'analyse, et l'intégration des données opérationnelles pour le succès du projet.</b>

# (Simple) Query the onto graph 

In [120]:
query = "Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?"
onto_query_engine = onto_index.as_query_engine(
 include_text=True,
 similarity_top_k=2,
)

response = onto_query_engine.query(query)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [121]:
display(Markdown(f"<b>{response}</b>"))

<b>Pour lancer un nouveau projet data avec vos données opérationnelles, vous pourriez impliquer Mme Sophie Laurent et M. Philippe Martin, car ils ont approuvé le compte rendu lié à la gestion des activités opérationnelles. Vous pourriez également envisager de travailler avec un rédacteur pour documenter le projet. Les étapes à suivre pourraient inclure un audit des données disponibles, le développement d'un accès personnalisé avec une interface d'utilisation, et l'organisation d'une session de formation pour l'utilisation des nouveaux outils.</b>

## (Node retriever)

In [ ]:
retriever = onto_index.as_retriever(
    include_text=True,  # include source text, default True
)

nodes = retriever.retrieve("Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?")


for node in nodes:
    print(node.text)


# Have a real chat with your data

## Set up the engines

### Graph engines

In [123]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    verbose=False,
)

### Onto engines

In [124]:
#onto_chat_engine = onto_index.query_engine(
#    chat_mode="condense_plus_context",
#    llm=llm
#)

In [126]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
graph_chat_engine = graph_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "Tu es un consultant spécialisé en stratégie et gouvernance des données."
        "Tu cites en priorité les métiers, les bases de données ou les champs de données présents dans ta base de connaissance."
        "Pour des idées de projets, tu vas proposer une stratégie et une gouvernance pour la mettre en oeuvre"
        "Tu feras attention à mettre en avant le Return On Investment (ROI) du projet proposé"
    ),
    verbose=False,
)

### Vector engines

In [122]:
memory = ChatMemoryBuffer.from_defaults(token_limit=3900)
vector_chat_engine = simple_index.as_chat_engine(
    chat_mode="condense_plus_context",
    memory=memory,
    llm=llm,
    context_prompt=(
        "Tu es un consultant spécialisé en stratégie et gouvernance des données."
        "Pour des idées de projets, tu vas proposer une stratégie et une gouvernance pour la mettre en oeuvre"
        "Tu feras attention à mettre en avant le Return On Investment (ROI) du projet proposé"
    ),
    verbose=False,
)

## Generate

In [125]:
vector_chat_engine.reset()

In [127]:
response_stream = vector_chat_engine.stream_chat("""D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible.Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?""")

INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible.Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?
Condensed question: D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible.Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?
Condensed question: D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible.Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?
Condensed question: D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible.Je dois lancer un nouveau projet data avec mes données opérationnelles, qui puis-je impliquer, et pour faire quoi ?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [128]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Lancer un projet data avec vos données opérationnelles est une initiative stratégique qui peut apporter une valeur significative à votre organisation. Voici une stratégie et une gouvernance pour mettre en œuvre ce projet, en mettant l'accent sur le ROI :

### 1. **Constitution de l'équipe projet**

#### a. **Sponsor Exécutif**
   - **Rôle** : Fournir un soutien stratégique et des ressources nécessaires. Assurer l'alignement du projet avec les objectifs de l'entreprise.
   - **ROI** : Garantit que le projet reste une priorité stratégique et reçoit le soutien nécessaire pour réussir.

#### b. **Chef de Projet Data**
   - **Rôle** : Gérer le projet au quotidien, c

In [129]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

In [130]:
section_modele = """ 


"""

In [131]:
response_stream = vector_chat_engine.stream_chat("""D'après tes connaissances et ta vision du monde, réponds moi aussi précisément que possible, quel serait le schéma de données idéal pour ce projet. Utilise le schéma de données standard Frictionless pour le décrire. Imite le style d'écriture de la {section_modele}""")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:llama_index.core.chat_engine.condense_plus_context:Condensed question: D'après tes connaissances et ta vision du monde, quel serait le schéma de données idéal pour un projet data utilisant des données opérationnelles, en utilisant le schéma de données standard Frictionless pour le décrire ? Peux-tu imiter le style d'écriture de la {section_modele} ?
Condensed question: D'après tes connaissances et ta vision du monde, quel serait le schéma de données idéal pour un projet data utilisant des données opérationnelles, en utilisant le schéma de données standard Frictionless pour le décrire ? Peux-tu imiter le style d'écriture de la {section_modele} ?
Condensed q

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [132]:
generate = response_stream.print_response_stream()


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Pour concevoir un schéma de données idéal pour votre projet en utilisant le standard Frictionless, il est essentiel de structurer vos données de manière à maximiser leur utilité et leur interopérabilité. Voici un exemple de schéma de données en utilisant le format Frictionless, qui est un standard ouvert pour la gestion des données tabulaires :

### {section_modele}

```json
{
  "profile": "tabular-data-package",
  "name": "operational-data-project",
  "resources": [
    {
      "name": "operations",
      "path": "data/operations.csv",
      "schema": {
        "fields": [
          {
            "name": "operation_id",
            "type": "integer",
         

In [133]:
display(Markdown(f"<b>{generate}</b>"))

<b>None</b>

## Sum-up

In [134]:
chat_history = memory.get_all()

# Assuming chat_history is available and contains your messages
assistant_messages = [
    message.content 
    for message in chat_history 
    if message.role == MessageRole.ASSISTANT  # Compare with the enum directly
]


output_filename = r"/Users/arthursarazin/Documents/coreandgraphs/graphandgovern/output116n87e.md"
# Write to a Markdown file
with open(output_filename, "w", encoding="utf-8") as f:
    for msg in assistant_messages:
        f.write(msg + "\n\n") 